In [3]:

import pandas as pd
import sqlite3
import json

# STEP 1: Load orders.csv (CSV Data)
orders = pd.read_csv(r"C:\Users\palak\Downloads\orders.csv")


# STEP 2: Load users.json (JSON Data)

with open(r"C:\Users\palak\Downloads\users.json", "r") as f:
    users = pd.DataFrame(json.load(f))


# STEP 3: Load restaurants.sql (SQL Data)

conn = sqlite3.connect(":memory:")

with open(r"C:\Users\palak\Downloads\restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


# STEP 4: Merge the Data (LEFT JOINS)

final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

# =========================================
# STEP 5: Save Final Dataset
# =========================================
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

# =========================================
# ANALYSIS SECTION
# =========================================

# Convert order_date to datetime
final_df["order_date"] = pd.to_datetime(final_df["order_date"])

# 1. Total orders by Gold members
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]

# 2. Total revenue from Hyderabad
hyderabad_revenue = round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)

# 3. Distinct users who placed at least one order
distinct_users = final_df["user_id"].nunique()

# 4. Average order value for Gold members
gold_avg_order_value = round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2
)

# 5. Orders placed for restaurants with rating >= 4.5
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]

# 6. Orders in top revenue city among Gold members
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_gold_city_orders = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

# =========================================
# PRINT RESULTS
# =========================================
print("Total orders by Gold members:", gold_orders_count)
print("Total revenue from Hyderabad:", hyderabad_revenue)
print("Distinct users:", distinct_users)
print("Average order value (Gold):", gold_avg_order_value)
print("Orders with rating >= 4.5:", high_rating_orders)
print("Top Gold revenue city:", top_gold_city)
print("Orders in top Gold revenue city:", top_gold_city_orders)


Total orders by Gold members: 4987
Total revenue from Hyderabad: 1889367
Distinct users: 2883
Average order value (Gold): 797.15
Orders with rating >= 4.5: 3374
Top Gold revenue city: Chennai
Orders in top Gold revenue city: 1337


C:\Users\palak\AppData\Local\Temp\ipykernel_29132\2776354560.py:45: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])
